# Working with the ScienceImage Class [v1]

In [32]:
# imports
import os
import numpy as np
from importlib import reload

In [3]:
# Path to PYPIT-Development-suite
pypdev_path = os.getenv('PYPIT_DEV')

----

# Development

In [80]:
from pypit.core import arprocimg
from pypit import biasframe
from pypit import flatfield
from pypit import processimages
from pypit import scienceimage
from pypit import traceslits
from pypit import waveimage
from pypit import wavetilts

## Settings

In [67]:
settings = dict(masters={})
settings['masters']['directory'] = pypdev_path+'/Cooked/MF_shane_kast_blue'
settings['masters']['reuse'] = True
settings['masters']['loaded'] = []
#
setup = 'A_01_aa'
#
settings['combine'] = {}

In [68]:
spectrograph = 'shane_kast_blue'
det = 1

In [75]:
settings['detector'] = {}
settings['detector']['num'] = det
settings['detector']['dataext'] = 0
settings['detector']['datasec01'] = [[0, 1024], [0, 0]]
settings['detector']['datasec02'] = [[1024, 2048], [0, 0]]
settings['detector']['oscansec01'] = [[2049, 2080], [0, 0]]
settings['detector']['oscansec02'] = [[2080, 2111], [0, 0]]
settings['detector']['naxis0'] = 2112  # Raw frame, with overscan
settings['detector']['naxis1'] = 350
settings['detector']['numamplifiers'] = 2
settings['detector']['gain'] = [1.2, 1.2]
settings['detector']['dispaxis'] = 1

## Load up

In [60]:
# Bias
settings['bias'] = {}
settings['bias']['useframe'] = 'bias'
#
biasFrame = biasframe.BiasFrame(setup=setup, settings=settings)
msbias = biasFrame.master()
msbias.shape

[WARNING] :: processimages.py 108 __init__() - No information on the spectrograph was given.  Do not attempt to (re)process the images
[INFO]    :: armasters.py 241 _load() - Loading a pre-existing master calibration frame


(2112, 350)

In [30]:
# Traceslits
settings.update(traceslits.default_settings().copy())
traceSlits = traceslits.TraceSlits(None, None, setup=setup, settings=settings)
traceSlits.master()

[INFO]    :: traceslits.py 520 _make_pixel_arrays() - Converting physical trace locations to nearest pixel
[INFO]    :: traceslits.py 527 _make_pixel_arrays() - Identifying the pixels belonging to each slit


True

In [13]:
# Wavelengths
waveImage = waveimage.WaveImage(setup=setup, settings=settings)
wave = waveImage.master()
wave.shape

[INFO]    :: armasters.py 241 _load() - Loading a pre-existing master calibration frame


(2048, 350)

In [17]:
# Tilts
waveTilts = wavetilts.WaveTilts(None, setup=setup, settings=settings)
tilts = waveTilts.master()
tilts.shape

[INFO]    :: armasters.py 241 _load() - Loading a pre-existing master calibration frame


(2048, 350)

In [62]:
# Flat
flatField = flatfield.FlatField(settings=settings, setup=setup)
msflat = flatField.master()

[WARNING] :: processimages.py 108 __init__() - No information on the spectrograph was given.  Do not attempt to (re)process the images
[INFO]    :: armasters.py 241 _load() - Loading a pre-existing master calibration frame


In [33]:
# Maskslits
maskslits = np.array([False])

In [49]:
# datasec image
reload(arprocimg)
datasec_img = damg.get_datasec_trimmed(
    spectrograph, None, settings['detector']['num'], settings['detector'],
    naxis0=settings['detector']['naxis0'],
    naxis1=settings['detector']['naxis1'])
datasec_img.shape

(2048, 350)

In [89]:
# BPM
bpm = np.zeros_like(datasec_img)

## Instantiate

In [34]:
files = [pypdev_path+'/RAW_DATA/Shane_Kast_blue/600_4310_d55/b27.fits.gz']

In [96]:
reload(scienceimage)
reload(scienceimage)

scienceImage = scienceimage.ScienceImage(file_list=files,
                                    spectrograph=spectrograph,
                                     settings=settings,
                                     tilts=tilts,
                                     det=det,
                                     tslits_dict=traceSlits.tslits_dict,
                                     pixlocn=traceSlits.pixlocn,
                                     datasec_img=datasec_img,
                                     maskslits=maskslits,
                                     setup=setup,
                                         bpm=bpm
                                    )

## Process

In [97]:
sciframe = scienceImage._process(msbias, msflat, apply_gain=True)

[INFO]    :: arload.py 327 load_raw_frame() - Loading raw_file: /home/xavier/local/Python/PYPIT-development-suite//RAW_DATA/Shane_Kast_blue/600_4310_d55/b27.fits.gz
[INFO]    :: processimages.py 268 bias_subtract() - Bias subtracting your image(s)
[INFO]    :: arprocimg.py 77 bias_subtract() - Subtracting bias image from raw frame


> /home/xavier/local/Python/PYPIT/pypit/processimages.py(241)apply_gain()
-> msgs.error("Need to input datasec_img!")
(Pdb) exit


BdbQuit: 

### View

In [92]:
sciframe.shape

(2048, 350)

In [66]:
settings['detector']

KeyError: 'detector'